In [1]:
import os
import logging
import json
from datetime import datetime
from pprint import pprint

# Import the verified_query module
from app.verified_query import (
    VerifiedQuery,
    Question,
    get_verified_query,
    get_verified_queries_by_vector_search,
    get_best_query,
    get_query_recommendations,
    get_follow_up_queries,
    modify_query
)
from app import config

# For database connection
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session

# For LLM service
from app.llm_service import LLMService

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure database connections
engine = create_engine(config.APPLICATION_DB_CONNECTION_STRING)
insurance_db_engine = create_engine(config.BUSINESS_DB_CONNECTION_STRING)

# Initialize LLM service
llm_service = LLMService()

/Users/avenugopal/.local/share/virtualenvs/smart_query_assistant-PYJH9x9M/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Testing test_query_recommendations

In [2]:
os.getenv("ANTHROPIC_API_KEY", "")

''

In [3]:
with Session(engine) as db:
    # First get a query to work with
    query_id = input("Enter a query ID to get recommendations for (or press Enter for default): ")
    if not query_id:
        query_id = "vq_top_agencies_premium"
    
    vq = get_verified_query(query_id, db)
    if not vq:
        print(f"No query found with ID: {query_id}")
    else:
    
        print(f"\nSelected query: {vq.name}")
        print(f"SQL:\n{vq.sql}")


Selected query: Top Performing Agencies by Premium
SQL:
SELECT 
  a.agency_id,
  a.agency_name,
  a.region,
  a.tier,
  COUNT(p.policy_id) AS policy_count,
  SUM(p.premium_amount) AS total_premium,
  AVG(p.premium_amount) AS avg_premium
FROM 
  agencies a
  JOIN agents ag ON a.agency_id = ag.agency_id
  JOIN policies p ON ag.agent_id = p.agent_id
WHERE 
  p.status = 'active' -- active, lapsed, cancelled
GROUP BY 
  a.agency_id, a.agency_name, a.region, a.tier
ORDER BY 
  total_premium DESC
LIMIT 10;


In [4]:
# Create context with user question
user_question = input("\nEnter user question for tailoring recommendations: ")
if not user_question:
    user_question = "Which agencies in my region generated the most premium in current quarter?"
    print(f"Using default question: '{user_question}'")

Using default question: 'Which agencies in my region generated the most premium in current quarter?'


In [5]:
# Add some sample context
context = {
    "calendar_context": "Current quarter: 2025 Q2, Previous quarter: 2025 Q1",
    "user_profile": "Region: Northeast, LOB: Personal Insurance",
}

In [6]:
print("\nGetting recommendations from LLM...")
recommendations = get_query_recommendations(vq, user_question, context, llm_service)

print("\nRecommendations:\n")
print(f"Modifications needed: {recommendations.get('modifications_needed', False)}")
print("\nExplanation:", recommendations.get('explanation', 'No explanation provided'))

if recommendations.get('modifications'):
    print("\nRecommended modifications:")
    for i, mod in enumerate(recommendations['modifications']):
        print(f"  {i+1}. Type: {mod.get('type')}")
        print(f"     Description: {mod.get('description')}")
        print(f"     SQL Impact: {mod.get('sql_impact')}")

2025-04-30 22:01:01,808 - app.verified_query - DEBUG - User prompt for LLM: SQL:
    SELECT 
  a.agency_id,
  a.agency_name,
  a.region,
  a.tier,
  COUNT(p.policy_id) AS policy_count,
  SUM(p.premium_amount) AS total_premium,
  AVG(p.premium_amount) AS avg_premium
FROM 
  agencies a
  JOIN agents ag ON a.agency_id = ag.agency_id
  JOIN policies p ON ag.agent_id = p.agent_id
WHERE 
  p.status = 'active' -- active, lapsed, cancelled
GROUP BY 
  a.agency_id, a.agency_name, a.region, a.tier
ORDER BY 
  total_premium DESC
LIMIT 10;

    Explanation:
    This query calculates the total premium amount generated by each agency.  It joins the agencies, agents, and policies tables to aggregate premium amounts for active policies, then sorts them by total premium in descending order and limits result to 10 records.


    This SQL query is designed to answer the following questions:
    [
  "Which agencies generate the most premium?",
  "What are our top agencies by premium revenue?",
  "Which ag


Getting recommendations from LLM...
Getting recommendations for tailoring the query
-------------->>>>>>>>>>>> Getting recommendations


2025-04-30 22:01:02,051 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 429, b'Too Many Requests', [(b'Date', b'Thu, 01 May 2025 03:01:02 GMT'), (b'Content-Type', b'application/json; charset=utf-8'), (b'Content-Length', b'337'), (b'Connection', b'keep-alive'), (b'vary', b'Origin'), (b'x-request-id', b'req_5c0d793038fd0254d52ae6cdf7079ad1'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=hicZrK2kYQGsuR4DvE7anqR5rv6Z3zBrrAvh9iLVu6s-1746068462-1.0.1.1-h6F_3vcZ6gofB3EZAUY8tCn6EFUJd.Sxm3.x7rpswy2pxDLRLaTH4seeZq6hJBK0GeE..saab9hDrb9NM6kv6GkQFlZylv5XypNgLbZe9_4; path=/; expires=Thu, 01-May-25 03:31:02 GMT; domain=.api.openai.com; HttpOnly; Secure; SameSite=None'), (b'X-Content-Type-Options', b'nosniff'), (b'Set-Cookie', b'_cfuvid=uY3T4PDjGoEG_jQx8r3JVXN3rBBVK1N.s0r9ipi_DXg-1746068462048-0.0.1.1-604800000; path=/; domain=.api.openai.com; HttpOnly; Secure; SameS

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [38]:
modsql = modify_query(vq.sql, recommendations['modifications'], llm_service)
print(modsql)

NameError: name 'modify_query' is not defined

In [8]:
system_prompt = """You are an expert SQL developer for PostgreSQL. Your task is to analyze and modify SQL based on specific requirements. Your response will be a valid SQL query."""

# User prompt with original SQL and modifications
user_prompt = f"""Original SQL:
        {vq.sql}

        Modification instructions:
        {recommendations['modifications']}

        Column Alias Guidelines:
        - Change only if necessary
        - Match the verb from user's question
        - Keep prefixes if present
        - Maintain quote style and capitalization

        Return only the modified SQL query. Do NOT include any other text or explanations.
        """

try:
    logger.info("Adjusting SQL query")
    
    # Get modified SQL from LLM
    modified_sql = llm_service.generate_text(
        prompt=user_prompt,
        system_prompt=system_prompt,
        temperature=0
    )
    
    modsql = modified_sql.strip()

    print("\nModified SQL:")
    print(modsql)

except Exception as e:
    print("Error generating modified SQL:", e)

2025-04-30 13:56:45,413 - __main__ - INFO - Adjusting SQL query
2025-04-30 13:56:45,414 - anthropic._base_client - DEBUG - Request options: {'method': 'post', 'url': '/v1/messages', 'timeout': Timeout(connect=5.0, read=600, write=600, pool=600), 'files': None, 'idempotency_key': 'stainless-python-retry-c4b1aaf6-ad5b-46b6-8e3d-22ba6ca388e0', 'json_data': {'max_tokens': 2000, 'messages': [{'role': 'user', 'content': 'Original SQL:\n        SELECT \n  a.agency_id,\n  a.agency_name,\n  a.region,\n  a.tier,\n  COUNT(p.policy_id) AS policy_count,\n  SUM(p.premium_amount) AS total_premium,\n  AVG(p.premium_amount) AS avg_premium\nFROM \n  agencies a\n  JOIN agents ag ON a.agency_id = ag.agency_id\n  JOIN policies p ON ag.agent_id = p.agent_id\nWHERE \n  p.status = \'active\' -- active, lapsed, cancelled\nGROUP BY \n  a.agency_id, a.agency_name, a.region, a.tier\nORDER BY \n  total_premium DESC\nLIMIT 10;\n\n        Modification instructions:\n        [{\'type\': \'filter\', \'description\': "


Modified SQL:
SELECT 
  a.agency_id,
  a.agency_name,
  COUNT(p.policy_id) AS policy_count,
  SUM(p.premium_amount) AS total_premium,
  AVG(p.premium_amount) AS avg_premium
FROM 
  agencies a
  JOIN agents ag ON a.agency_id = ag.agency_id
  JOIN policies p ON ag.agent_id = p.agent_id
WHERE 
  p.status = 'active'
  AND a.region = 'Northeast'
  AND p.start_date BETWEEN '2025-04-01' AND '2025-06-30'
GROUP BY 
  a.agency_id, a.agency_name
ORDER BY 
  total_premium DESC
LIMIT 10;


In [ ]:
# Execute modified SQL 
with Session(insurance_db_engine) as db:
        # Run the query 
        try:
            result = db.execute(text(modsql))
            rows = result.fetchall()
            columns = result.keys()
            data = [dict(zip(columns, r)) for r in rows]
            # Convert to DataFrame for better visualization
            df_data = pd.DataFrame(data)
        except Exception as e:
            print("Error executing query:", e)     
df_data           

,agency_id,agency_name,policy_count,total_premium,avg_premium
0,7,Capital Benefits Agency,1,1500.00,1500.0000000000000000


In [ ]:
with Session(engine) as db:
    # Get all query IDs first to show what's available
    result = db.execute(text("SELECT * FROM verified_query"))
    queries = result.fetchall()
    

    # Pandas dataframe for better visualization
    import pandas as pd
    df = pd.DataFrame(queries)

    print("Available queries in database:")
    display(df)

Available queries in database:


,id,name,query_explanation,sql,instructions,tables_used,verified_at,verified_by
0,vq_top_agencies_premium,Top Performing Agencies by Premium,This query calculates the total premium amount...,"SELECT \n a.agency_id,\n a.agency_name,\n a...",Modification guidelines: 1. Filter conditions:...,"[agencies, agents, policies]",2025-04-15 10:00:00,data_analyst
1,vq_distribution_channel_performance,Distribution Channel Performance Analysis,This query analyzes the performance of differe...,"SELECT \n dc.channel_id,\n dc.channel_name,\...",Modification guidelines: 1. Filter conditions:...,"[distribution_channels, policies]",2025-04-15 14:30:00,data_analyst
2,vq_agent_performance_report,Agent Performance Report,This query provides a performance report for a...,"SELECT \n ag.agent_id,\n ag.agent_name,\n a...",Modification guidelines: 1. Filter conditions:...,"[agents, agencies, policies]",2025-04-15 15:45:00,data_analyst
3,vq_claims_analysis_by_policy_type,Claims Analysis by Policy Type,This query analyzes claims data grouped by pol...,"SELECT \n p.policy_type,\n COUNT(p.policy_id...",Modification guidelines: 1. Filter conditions:...,"[policies, claims]",2025-04-16 09:15:00,data_analyst
4,vq_agency_performance_by_region,Regional Agency Performance,This query analyzes agency performance by regi...,"SELECT \n a.region,\n COUNT(DISTINCT a.agenc...",Modification guidelines: 1. Filter conditions:...,"[agencies, agents, policies]",2025-04-16 11:30:00,data_analyst
5,vq_agency_performance_by_tier,Agency Performance by Tier,This query analyzes agency performance by tier...,"SELECT \n a.tier,\n COUNT(DISTINCT a.agency_...",Modification guidelines: 1. Filter conditions:...,"[agencies, agents, policies]",2025-04-16 13:45:00,data_analyst
6,vq_top_agencies_avg_premium,Top Agencies by Average Premium,This query identifies agencies with the highes...,"SELECT \n a.agency_id,\n a.agency_name,\n a...",Modification guidelines: 1. Filter conditions:...,"[agencies, agents, policies]",2025-04-17 09:00:00,data_analyst
7,vq_policy_type_performance,Policy Type Performance Analysis,This query analyzes the performance of differe...,"SELECT \n p.policy_type,\n COUNT(p.policy_id...",Modification guidelines: 1. Filter conditions:...,[policies],2025-04-17 11:15:00,data_analyst
8,vq_channel_growth_over_time,Distribution Channel Growth Analysis,This query analyzes the growth of different di...,"SELECT \n date_trunc('quarter', p.start_date)...",Modification guidelines: 1. Filter conditions:...,"[distribution_channels, policies]",2025-04-17 14:30:00,data_analyst
9,vq_channel_commission_analysis,Distribution Channel Commission Analysis,This query analyzes commission payouts across ...,"SELECT \n dc.channel_id,\n dc.channel_name,\...",Modification guidelines: 1. Filter conditions:...,"[distribution_channels, policies]",2025-04-18 09:45:00,data_analyst


## Validating all queries by running them

In [4]:
print("Total queries in database:", len(df))
for index, row in df.iterrows():
    print(f"Validating [{(index+1):03}]: {row['id']} ({row['name']})")
    with Session(insurance_db_engine) as db:
        # Run the query 
        try:
            result = db.execute(text(row['sql']))
            rows = result.fetchall()[:1]  # Limit to first 10 rows for display
            columns = result.keys()
            data = [dict(zip(columns, r)) for r in rows]
            # Convert to DataFrame for better visualization
            df_data = pd.DataFrame(data)
        except Exception as e:
            print("Error executing query:", e)


Total queries in database: 11
Validating [001]: vq_top_agencies_premium (Top Performing Agencies by Premium)
Validating [002]: vq_distribution_channel_performance (Distribution Channel Performance Analysis)
Validating [003]: vq_agent_performance_report (Agent Performance Report)
Validating [004]: vq_claims_analysis_by_policy_type (Claims Analysis by Policy Type)
Validating [005]: vq_agency_performance_by_region (Regional Agency Performance)
Validating [006]: vq_agency_performance_by_tier (Agency Performance by Tier)
Validating [007]: vq_top_agencies_avg_premium (Top Agencies by Average Premium)
Validating [008]: vq_policy_type_performance (Policy Type Performance Analysis)
Validating [009]: vq_channel_growth_over_time (Distribution Channel Growth Analysis)
Validating [010]: vq_channel_commission_analysis (Distribution Channel Commission Analysis)
Validating [011]: vq_agent_certification_performance (Agent Performance by Certification Level)


## Testing get_verified_query

In [5]:
with Session(engine) as db:
    # Ask for query ID to test
    query_id = input("\nEnter a query ID to retrieve (or press Enter for 'vq_top_agencies_premium'): ")
    if not query_id:
        query_id = "vq_top_agencies_premium"
    
    # Get the query
    vq = get_verified_query(query_id, db)
    
    if vq:
        print(f"\nRetrieved query: {vq.name}")
        print(f"Explanation: {vq.query_explanation[:100]}...")
        print(f"SQL:\n{vq.sql[:200]}...")
        print(f"Number of questions: {len(vq.questions)}")
        if vq.questions:
            print("Sample questions:")
            for i, q in enumerate(vq.questions[:3]):
                print(f"  {i+1}. {q.text}")
        print(f"Follow-ups: {vq.follow_ups}")
    else:
        print(f"No query found with ID: {query_id}")


Retrieved query: Top Performing Agencies by Premium
Explanation: This query calculates the total premium amount generated by each agency.  It joins the agencies, age...
SQL:
SELECT 
  a.agency_id,
  a.agency_name,
  a.region,
  a.tier,
  COUNT(p.policy_id) AS policy_count,
  SUM(p.premium_amount) AS total_premium,
  AVG(p.premium_amount) AS avg_premium
FROM 
  agencies a
...
Number of questions: 4
Sample questions:
  1. Which agencies generate the most premium?
  2. What are our top agencies by premium revenue?
  3. Which agencies have the highest premium amounts?
Follow-ups: ['vq_agency_performance_by_region', 'vq_agency_performance_by_tier', 'vq_top_agencies_avg_premium', 'vq_policy_type_performance', 'vq_claims_analysis_by_policy_type', 'vq_distribution_channel_performance', 'vq_agent_performance_report']


## Testing get_verified_queries_by_vector

In [7]:
with Session(engine) as db:
    # Test with a few different questions
    test_questions = [
        "Which agencies generate the most premium?",
        "How are our distribution channels performing?",
        "What is the average claim amount for auto policies?",
        "Show me agency performance by region"
    ]
    
    print("Test questions:")
    for i, q in enumerate(test_questions):
        print(f"  {i+1}. {q}")
    
    # Ask for question or custom input
    question_num = input("\nEnter question number to use (1-4), or type your own question: ")
    
    try:
        idx = int(question_num) - 1
        if 0 <= idx < len(test_questions):
            question = test_questions[idx]
        else:
            question = question_num
    except ValueError:
        question = question_num
    
    print(f"\nSearching for: '{question}'")
    
    # Get results
    results = get_verified_queries_by_vector_search(question, n=3, db=db)
    
    print(f"\nFound {len(results)} results:")
    for i, result in enumerate(results):
        vq = result["verified_query"]
        print(f"\nResult {i+1}:")
        print(f"  Query: {vq.name} (ID: {vq.id})")
        print(f"  Similarity: {result['similarity']:.4f}")
        print(f"  Matched question: '{result['matched_question']}'")
        print(f"  Explanation: {vq.query_explanation[:100]}...")

Test questions:
  1. Which agencies generate the most premium?
  2. How are our distribution channels performing?
  3. What is the average claim amount for auto policies?
  4. Show me agency performance by region

Searching for: 'which are the best agencies?'


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.93it/s]


    SELECT 
        vq.id, 
        1 - (q.vector_embedding <=> '[0.048875093,-0.13988926,-0.1026287,0.0016150457,0.009610684,0.025216253,0.003057069,0.0033119167,0.014516376,0.016063794,-0.02720114,0.027853236,-0.033775564,0.06956033,0.026320929,-0.0138694495,0.08349392,0.02796839,0.033016093,-0.11185863,-0.05996178,0.007484275,0.02785974,-0.046576846,0.009911235,0.003370585,-0.092383794,-0.058274582,-0.05668949,-0.044252142,-0.04337034,0.0016410353,-0.033561863,0.014316582,0.09615959,0.076091595,-0.030655498,-0.043862708,0.054361947,-0.009110285,-0.0313952,0.024144145,0.01151092,-0.08001156,0.0023916108,-0.11548402,0.031824633,0.07836838,-0.044699833,0.08363068,-0.011621376,-0.07680655,-0.005733484,0.08937619,-0.036168717,0.02943545,-0.07103481,-0.011277833,-0.05824684,-0.044279188,-0.09186385,0.0070578144,-0.02128941,0.034570713,-0.0008568118,0.07486578,-0.05636881,0.0466204,-0.033040926,-0.1127936,-0.03359681,-0.15078236,-0.06428328,0.047789697,0.105715655,0.03705486,0.05166197,0.

## Testing get_best_query

This uses LLM to work through matched queries and select the best

In [8]:
with Session(engine) as db:
    # Test with a complex question
    question = input("Enter a question to find the best query for: ")
    if not question:
        question = "Which agencies in the Northeast region had the highest premium last quarter?"
        print(f"Using default question: '{question}'")
    else:
        print(f"Searching for: '{question}'")
    
    # Get results
    print("\nFinding best matching query using vector search + LLM ranking...")
    result = get_best_query(question, llm_service, db)
    
    if result:
        vq = result["verified_query"]
        print(f"\nBest match: {vq.name} (ID: {vq.id})")
        print(f"Similarity score: {result['similarity']:.4f}")
        print(f"Confidence: {result.get('confidence', 'N/A')}")
        print(f"Matched question: '{result.get('matched_question', 'N/A')}'")
        print(f"Reasoning: {result.get('reasoning', 'N/A')}")
        print(f"\nQuery explanation: {vq.query_explanation}")
        print(f"\nSQL:\n{vq.sql}")
    else:
        print("No matching query found")

Searching for: 'who is the second best agency'

Finding best matching query using vector search + LLM ranking...


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.60it/s]


    SELECT 
        vq.id, 
        1 - (q.vector_embedding <=> '[0.0015647069,-0.13904984,-0.12597209,-0.07016162,0.02423532,0.055151895,0.048906732,0.051763356,0.0613916,0.018893952,-0.037277535,-0.00085522735,0.0026301285,0.07665519,0.014043694,0.0154475905,0.06005047,0.0672471,0.02573232,-0.106669895,0.005118747,-0.032322697,0.0029294472,-0.05729535,0.012893771,0.029693015,-0.07562187,0.07719659,-0.063048795,-0.065049954,-0.017976124,-0.029095642,-0.022287395,0.032376923,0.037246715,0.061581008,-0.08828067,-0.023695545,0.06825991,-0.015273034,0.0067560235,-0.013836501,-0.011125074,-0.10917138,-0.027082682,-0.09611595,0.03858806,0.04049926,-0.039338995,0.07651062,-0.023089198,-0.020769428,0.0023538605,0.008659533,0.026048547,0.12109438,-0.08971686,-0.015856713,-0.05554514,-0.038599763,0.015686952,0.00035926892,-0.009719041,0.032915384,0.0450306,-0.032249454,-0.06823886,-0.018104654,-0.09330339,-0.114240006,0.020233855,-0.10615345,-0.06255493,-0.020708771,0.09737037,0.018709932,0.03


2025-04-30 12:08:32,442 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



Best match: Top Performing Agencies by Premium (ID: vq_top_agencies_premium)
Similarity score: 0.5682
Confidence: 0.8
Matched question: 'Show me the agencies with the most premium'
Reasoning: The user's question 'who is the second best agency' is best matched by the 'Top Performing Agencies by Premium' query. This query provides a ranking of agencies by total premium, which can be used to identify the second best agency. The other candidate queries focus on agent performance, agency tiers, and regional comparisons, which do not directly address the specific question asked.

Query explanation: This query calculates the total premium amount generated by each agency.  It joins the agencies, agents, and policies tables to aggregate premium amounts for active policies, then sorts them by total premium in descending order and limits result to 10 records.


SQL:
SELECT 
  a.agency_id,
  a.agency_name,
  a.region,
  a.tier,
  COUNT(p.policy_id) AS policy_count,
  SUM(p.premium_amount) AS tota

## Test getting follow-up queries


In [10]:
with Session(engine) as db:
    # Get a query with follow-ups
    query_id = input("Enter a query ID to get follow-ups for (or press Enter for default): ")
    if not query_id:
        query_id = "vq_top_agencies_premium"
    
    print(f"\nGetting follow-ups for query ID: {query_id}")
    follow_ups = get_follow_up_queries(query_id, db)
    
    if follow_ups:
        print(f"\nFound {len(follow_ups)} follow-up queries:")
        for i, fu in enumerate(follow_ups):
            print(f"\nFollow-up {i+1}: {fu.name} (ID: {fu.id})")
            print(f"Explanation: {fu.query_explanation[:100]}...")
            print(f"Questions: {', '.join([q.text for q in fu.questions[:2]])}")
    else:
        print(f"No follow-up queries found for query ID: {query_id}")


Getting follow-ups for query ID: vq_top_agencies_premium

Found 7 follow-up queries:

Follow-up 1: Regional Agency Performance (ID: vq_agency_performance_by_region)
Explanation: This query analyzes agency performance by region, showing the number of agencies, average premium pe...
Questions: How are our agencies performing by region?, Which regions generate the most premium?

Follow-up 2: Agency Performance by Tier (ID: vq_agency_performance_by_tier)
Explanation: This query analyzes agency performance by tier (Gold, Silver, Bronze, Platinum), showing the number ...
Questions: How are agencies performing by tier?, Which agency tier generates the most premium?

Follow-up 3: Top Agencies by Average Premium (ID: vq_top_agencies_avg_premium)
Explanation: This query identifies agencies with the highest average premium per policy, which may indicate agenc...
Questions: Which agencies have the highest average premium per policy?, What agencies focus on high-value policies?

Follow-up 4: Polic